In [0]:
root_folder = '/content/drive/My Drive/Colab Notebooks/SMAI/2019201089-Assignment 2/Datasets/Question-6'
import os
os.chdir(root_folder)

In [2]:
!ls
!apt install unzip
!unzip dataset.zip

1  2  3  4  5  dataset	dataset.zip
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Archive:  dataset.zip
  inflating: dataset/243_1.txt       
  inflating: dataset/239_1.txt       
  inflating: dataset/234_1.txt       
  inflating: dataset/233_1.txt       
  inflating: dataset/229_1.txt       
  inflating: dataset/226_1.txt       
  inflating: dataset/223_1.txt       
  inflating: dataset/222_1.txt       
  inflating: dataset/221_1.txt       
  inflating: dataset/220_1.txt       
  inflating: dataset/218_1.txt       
  inflating: dataset/215_1.txt       
  inflating: dataset/208_1.txt       
  inflating: dataset/207_1.txt       
  inflating: dataset/206_1.txt       
  inflating: dataset/205_1.txt       
  inflating: dataset/203_1.txt       
  inflating: dataset/201_1.txt       
  inflating: dataset/200_1.txt       
  inflat

In [3]:
!ls dataset

100_1.txt  144_2.txt  188_5.txt  233_3.txt  277_4.txt  345_4.txt  55_4.txt
100_2.txt  144_3.txt  189_1.txt  233_4.txt  277_5.txt  34_5.txt   55_5.txt
100_3.txt  144_4.txt  189_2.txt  233_5.txt  278_1.txt  346_1.txt  5_5.txt
100_4.txt  144_5.txt  189_3.txt  23_3.txt   278_2.txt  346_4.txt  56_1.txt
100_5.txt  14_4.txt   189_4.txt  234_1.txt  278_3.txt  347_1.txt  56_2.txt
101_1.txt  145_1.txt  189_5.txt  234_2.txt  278_4.txt  347_4.txt  56_3.txt
101_2.txt  145_2.txt  190_1.txt  234_3.txt  278_5.txt  348_1.txt  56_4.txt
101_3.txt  145_3.txt  190_2.txt  234_4.txt  279_1.txt  348_4.txt  56_5.txt
101_4.txt  145_4.txt  190_3.txt  234_5.txt  279_2.txt  349_1.txt  57_1.txt
101_5.txt  145_5.txt  190_4.txt  23_4.txt   279_3.txt  349_4.txt  57_2.txt
10_1.txt   14_5.txt   190_5.txt  235_1.txt  279_4.txt  3_4.txt	  57_3.txt
102_1.txt  146_1.txt  191_1.txt  235_2.txt  279_5.txt  350_1.txt  57_4.txt
102_2.txt  146_2.txt  191_2.txt  235_3.txt  280_1.txt  350_4.txt  57_5.txt
102_3.txt  146_3.txt  191_3

In [4]:
!mkdir 1 2 3 4 5

mkdir: cannot create directory ‘1’: File exists
mkdir: cannot create directory ‘2’: File exists
mkdir: cannot create directory ‘3’: File exists
mkdir: cannot create directory ‘4’: File exists
mkdir: cannot create directory ‘5’: File exists


In [0]:
!mv dataset/[^_]*_1.txt 1/
!mv dataset/[^_]*_2.txt 2/
!mv dataset/[^_]*_3.txt 3/
!mv dataset/[^_]*_4.txt 4/
!mv dataset/[^_]*_5.txt 5/

In [6]:
!ls

1  2  3  4  5  dataset	dataset.zip


##Imports

In [7]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.metrics import homogeneity_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.datasets 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


###Feature USED
TFIDF
Term frequency: Number of time a word occurs in a document is coined as TF
$$tf = \#word-frequenct$$

INV DOC Freq : log of (number of documents/ total number of occurances of the word in all documnets together)
$$idf = \log(\#documents / \sum\#tf)$$

###Lemmatization:
Use of lemmatization elimates similar kind of words like working, works all map to work.

###Stop Words Removal
I have remove stopwords which are generally recurring words in english like the, is, this, am, was, were...

### Use of min_df 
Will eliminate word which are less the certain freqency in whole dataset, most probable they are outliers

## kmeans

In [0]:
class Cluster:
    def __init__(self, min_df=3, ngram_range=(1,2), stop_words='english', epochs=100, p=2, k=5):
        '''
        model atts
        k for k means
        model to hold model
        '''
        self.k = k
        self.train_x = self.train_y = None
        self.kmeans = None
        self.epochs = epochs
        self.p = p
        '''
        Preprocessing attrs
        '''
        self.tfidf_vectorizer = None
        self.min_df = min_df
        self.ngram_range = ngram_range
        self.stop_words = stop_words

    def pre_process(self, data_set):
        '''
        Pre process the data frame
        '''

        class lemma_tokenizer:
            '''
            Lemmathization class to be passed as argument of TFIDF while preprocessing
            '''
            def __init__(self):
                '''
                Used for Wordnet lemmatizer initialization
                '''
                self.word_net_lemmatizer = WordNetLemmatizer()
            def __call__(self, docs):
                return [self.word_net_lemmatizer.lemmatize(token) for token in word_tokenize(docs)]

        text_data = data_set.data
        print('Doing Pre Processing')
        self.tfidf_vectorizer = TfidfVectorizer(min_df=self.min_df, ngram_range=self.ngram_range, stop_words=self.stop_words, tokenizer=lemma_tokenizer())

        print('\n\n\n', self.tfidf_vectorizer, '\n\n\n')
        
        processed_x = self.tfidf_vectorizer.fit_transform(text_data).A
        # processed_x = sparse.csr_matrix(self.tfidf_vectorizer.transform(text_data).A)

        processed_x = np.append(processed_x, np.array([-1] * processed_x.shape[0]).reshape((processed_x.shape[0], 1)), axis=1)
        print('\n\n\n', 'Finished Preprocessing','\n\n\n')
        return processed_x

    def random_center_initialize(self):
        '''
        Random initialization of k means for initial guess centers/centroids
        '''
        self.kmeans = self.train_x[np.random.randint(low=0, high=self.train_x.shape[0]-1, size=self.k)][:,:-1]

    def _cluster(self):
        import copy
        previous_cluster = self.train_x[:, -1].copy()
        convergence_count = 0
        for epoch in range(self.epochs):
            convergence_count = epoch
            if epoch%10 == 0:
                print('Epoch', epoch)
            for i in range(self.train_x.shape[0]):
                distance = (((self.kmeans - self.train_x[i, :-1])**self.p).sum(axis=1))**(1/self.p)
                self.train_x[i, -1] = distance.argmin()
                
            if np.array_equal(previous_cluster, self.train_x[:, -1]):
                '''
                converged so break the loop no point of continuing epochs
                '''
                break
            previous_cluster = self.train_x[:, -1].copy()

            for i in range(self.k):
                self.kmeans[i,:] = self.train_x[self.train_x[:, -1]==i][:,:-1].mean(axis=0)

        return convergence_count

    def cluster(self, train_path='./cluster_dataset'):
        train_path = train_path+'/..'
        data_set = sklearn.datasets.load_files(train_path, encoding='utf-8', decode_error='ignore')
        
        import re
        for i in range(data_set.data.__len__()):
            data_set.data[i] = re.sub("[^A-Za-z']+", ' ', str(data_set.data[i])).lower()
        # print(data_set.data.__len__(), data_set.data[0])
        
        self.train_x = self.pre_process(data_set)

        self.random_center_initialize()

        self._cluster()

        clusters = dict([(i.split('/')[-1],[]) for i in data_set.filenames])
        for j,i in zip(self.train_x[:,-1], data_set.filenames):
            clusters[i.split('/')[-1]].append(j)
        
        print('Homogenity Score\t\t\t\t', '{0:.2f}'.format(homogeneity_score(data_set.target, self.train_x[:,-1])))
        return clusters

In [9]:
# from q6 import Cluster as cl
cluster_algo = Cluster(k=5)
# You will be given path to a directory which has a list of documents. You need to return a list of cluster labels for those documents
predictions = cluster_algo.cluster('./dataset/') 


Doing Pre Processing



 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.Cluster.pre_process.<locals>.lemma_tokenizer object at 0x7faf990a1908>,
                use_idf=True, vocabulary=None) 





/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))





 Finished Preprocessing 



Epoch 0
Homogenity Score				 0.64


In [10]:
for i in predictions.keys():
    print(i,predictions[i][0],'\n')

271_5.txt 4.0 

212_1.txt 2.0 

57_2.txt 1.0 

325_1.txt 2.0 

256_5.txt 4.0 

296_5.txt 4.0 

17_3.txt 4.0 

44_4.txt 3.0 

293_4.txt 3.0 

75_3.txt 0.0 

106_1.txt 2.0 

265_1.txt 2.0 

287_5.txt 4.0 

48_4.txt 3.0 

199_4.txt 1.0 

291_3.txt 4.0 

207_1.txt 2.0 

16_1.txt 2.0 

334_1.txt 2.0 

109_5.txt 3.0 

27_4.txt 3.0 

62_3.txt 0.0 

295_5.txt 4.0 

311_3.txt 0.0 

223_2.txt 1.0 

72_1.txt 2.0 

317_1.txt 2.0 

370_1.txt 4.0 

166_2.txt 1.0 

170_2.txt 4.0 

274_5.txt 4.0 

252_2.txt 1.0 

176_5.txt 4.0 

97_2.txt 1.0 

66_5.txt 4.0 

23_2.txt 4.0 

259_1.txt 2.0 

57_5.txt 4.0 

131_1.txt 2.0 

173_1.txt 2.0 

285_4.txt 3.0 

168_4.txt 3.0 

366_4.txt 3.0 

278_5.txt 4.0 

329_4.txt 1.0 

57_4.txt 3.0 

72_4.txt 3.0 

315_3.txt 0.0 

409_4.txt 3.0 

24_3.txt 4.0 

221_4.txt 3.0 

289_3.txt 0.0 

143_1.txt 2.0 

231_4.txt 1.0 

359_4.txt 3.0 

128_2.txt 1.0 

33_5.txt 4.0 

149_1.txt 2.0 

227_5.txt 4.0 

297_3.txt 0.0 

155_4.txt 3.0 

192_2.txt 4.0 

9_4.txt 3.0 

193_2.txt 1